In [7]:
# Description: Job Shop Scheduling Problem using Constraint Programming
# import the necessary classes
import pandas as pd
import numpy as np
import time
from ortools.constraint_solver import pywrapcp
import collections
from ortools.sat.python import cp_model
from ortools.constraint_solver.pywrapcp import IntervalVar 
from ortools.init import pywrapinit
import pandas as pd 


In [8]:
def extract_data(filename):
    # Initialize variables to store the data.
    nj = []
    nm = []
    times = []
    machines = []


    # Open the file and read each line.
    with open(filename, 'r') as file:
        contents = file.read()
        lines = contents.splitlines()
        # Iterate through each line to find the number of jobs and machines.
        for line in range(len(lines)):
            # Check if the line contains the number of jobs and machines.
            if 'Nb of jobs' in lines[line]:
                nj.append(lines[line + 1].split()[0])
                nm.append(lines[line + 1].split()[1])
            break
        num_jobs = int(nj[0])
        num_machines = int(nm[0])
        for line in range(len(lines)):
            # Check if the line contains the number of jobs and machines.
            if 'Nb of jobs' in lines[line]:
                continue
            if ' ' in lines[line]:
                continue
            elif 'Times' in lines[line]:
                pos = int(0)
                while pos < num_jobs:
                    pos += 1
                    times.append(lines[line + pos].split())
        for line in range(len(lines)):
            if 'Nb of jobs' in lines[line]:
                continue
            if ' ' in lines[line]:
                continue
            if 'Times' in lines[line]:
                continue
            elif 'Machines' in lines[line]:
                pos = int(0)
                while pos < num_jobs:
                    pos +=1
                    machines.append(lines[line + pos].split())
        # Having extracted the data, we now need to convert the data into a format that can be used by the solver.
        # We will convert the data into a list of lists.
        # The first list will contain the number of jobs and the number of machines.
        # The second list will contain the processing times.
        # The third is a list of the combination of jobs and machines.
        # The final input will be a list of lists in the respective order required by the solver.
            # Desired output
                ### List - 1º Level  - Size 10
                ### List - 2º Level  - Size 50
                ### List - 3º Level  - Size 15
                ### List - 4º Level  - Size 2

        list1 = times
        list2 = machines
        list = [(int(x), int(y)) for sublist1, sublist2 in zip(list2, list1) for x, y in zip(sublist2, sublist1)]
        new_list = [i for i in range(1, len(list)+1)]
    
        for i, (x, y) in enumerate(list):
            new_list[i] = [(x, y)]
        num_elements = num_jobs*num_machines
        small_lists = [new_list[i:i + num_elements] for i in range(0, len(new_list), num_elements)]
        smaller_lists = [i for i in range(len(small_lists))]
        for i in range(len(small_lists)):
            smaller_lists[i] = [small_lists[i][j:j + num_machines] for j in range(0, len(small_lists[i]), num_machines)]
        for i in range(len(smaller_lists)):
            for j in range(len(smaller_lists[i])):
                for k in range(len(smaller_lists[i][j])):
                    for l in range(len(smaller_lists[i][j][k])):
                        smaller_lists[i][j][k] = smaller_lists[i][j][k][l]
                        smaller_lists[i][j][k] = tuple(reversed(smaller_lists[i][j][k]))
        return ( num_jobs, num_machines,smaller_lists)

In [11]:
num_jobs, num_machines, smaller_lists = extract_data('tai15_15.txt')

In [14]:
# Check the size of the list if it is the correct size as expected for the file selected.
print(len(smaller_lists))
print(len(smaller_lists[0]))
print(len(smaller_lists[0][0]))
print(len(smaller_lists[0][0][0]))

10
15
15
2


In [5]:
def main(jobs_data):
    """Entry point of the program."""
    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(1,machines_count)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model.
    model = cp_model.CpModel()

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var,
                                                   end=end_var,
                                                   interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id +
                                1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 60.0 
    solver.parameters.num_search_workers = 8
    solver.parameters.log_search_progress = True
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solution:')
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(start=solver.Value(
                        all_tasks[job_id, task_id].start),
                                       job=job_id,
                                       index=task_id,
                                       duration=task[1]))

        # Create per machine output lines.
        output = ''
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_task_%i' % (assigned_task.job,
                                           assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-15s' % name

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                sol_line += '%-15s' % sol_tmp

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
        print(output)
    else:
        print('No solution found.')

    # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())
    print('  - status   : %s' % solver.StatusName(status))
    print('  - objective: %i' % solver.ObjectiveValue())
    print('  - User Time  : %i' % solver.UserTime())

In [6]:
list_files = ['tai15_15.txt','tai20_15.txt','tai20_20.txt','tai30_15.txt','tai30_20.txt','tai50_15.txt','tai50_20.txt','tai100_20.txt']
start_cell_time = time.time()
for i in range(len(list_files)):
    start_list_time = time.time()
    print(f'This is the file: {list_files[i]}')
    num_jobs, num_machines, smaller_lists = extract_data(list_files[i])
    for i in range(len(smaller_lists)):
        start_time = time.time()
        print (f'This is the list number: {i}')
        
        main(smaller_lists[i])
        print("--- %.2f seconds ---" % (time.time() - start_time))
    print("--- %.2f seconds for the file---" % (time.time() - start_list_time))
print("--- %.2f seconds for the cell---" % (time.time() - start_cell_time))


This is the file: tai15_15.txt
This is the list number: 0
Solution:
Optimal Schedule Length: 1231.0
Machine 1: job_7_task_2   job_11_task_2  job_9_task_5   job_10_task_3  job_8_task_4   job_3_task_5   job_12_task_7  job_2_task_8   job_14_task_9  job_5_task_11  job_1_task_10  job_6_task_12  job_4_task_13  job_0_task_13  job_13_task_14 
           [84,93]        [93,168]       [222,239]      [262,270]      [309,322]      [336,430]      [593,670]      [786,840]      [851,859]      [866,953]      [962,980]      [993,1030]     [1031,1073]    [1073,1143]    [1187,1222]    
Machine 2: job_2_task_0   job_8_task_5   job_14_task_5  job_7_task_7   job_6_task_6   job_13_task_7  job_9_task_10  job_11_task_11 job_4_task_10  job_5_task_9   job_12_task_10 job_10_task_11 job_3_task_13  job_1_task_13  job_0_task_14  
           [0,4]          [322,409]      [415,441]      [441,503]      [506,525]      [525,581]      [583,652]      [652,697]      [697,790]      [790,816]      [816,864]      [864,957]    